## Tokenizer usage

A notebook for experimenting with and understanding tokenization and its options, including featurization.

First, some initializations...

In [1]:
import latok.core.default_tokenizer as tokenizer
import pandas as pd

### Parameters

**Edit the following parameters as desired**, including the text to tokenize, before running the next cell...

In [2]:
text = '''This is a1 test don't http://foo.com?bar=123 @user abc@xyz.com camelCaseOne, CamelCaseTwo, camelCase1, CamelCase2, 123 $123,456.78'''


# Abstract features:
#  A list of FeatureSpec instances to use for abstract featurization
#  (Refer to "abstract featurization" description below)
feature_specs = [
    tokenizer.TWITTER_FEATURE,
    tokenizer.EMAIL_FEATURE,
    tokenizer.URL_FEATURE,
    tokenizer.CAMEL_CASE_FEATURE,
    tokenizer.NUMERIC_FEATURE,
    tokenizer.EMBEDDED_APOS_FEATURE,
]

### Basic Tokenization

tokenizer.tokenize generates each token.

We'll wrap each token for placement in a pandas DataFrame for display...

In [3]:
token_texts = [(token_text) for token_text in tokenizer.tokenize(text)]

print(f'Basic tokenization of text:\n\n{text}\n')

df = pd.DataFrame(token_texts, columns=['token'])
df

Basic tokenization of text:

This is a1 test don't http://foo.com?bar=123 @user abc@xyz.com camelCaseOne, CamelCaseTwo, camelCase1, CamelCase2, 123 $123,456.78



,token
0,This
1,is
2,a1
3,test
4,don't
5,http://foo.com?bar=123
6,@user
7,abc@xyz.com
8,camel
9,Case


### LaToken object generation

Instead of just token text, a LaToken object can be generated for each token using the ```featurize``` method.

This, among other things, preserves the original character locations of the tokens.

First, build and populate the token objects to be displayed below...

In [4]:
la_tokens = list(tokenizer.featurize(text))
tokenizer.add_abstract_features(la_tokens, feature_specs)

Show the basic objects...

In [5]:
print(f'Object tokenization of text:\n\n{text}\n')

data = [(token.text, token.start_idx, token.end_idx) for token in la_tokens]
df = pd.DataFrame(data, columns=['token', 'start_idx', 'end_idx'])
df

Object tokenization of text:

This is a1 test don't http://foo.com?bar=123 @user abc@xyz.com camelCaseOne, CamelCaseTwo, camelCase1, CamelCase2, 123 $123,456.78



,token,start_idx,end_idx
0,This,0,4
1,is,4,7
2,a1,7,10
3,test,10,15
4,don't,15,21
5,http://foo.com?bar=123,21,44
6,@user,44,50
7,abc@xyz.com,50,62
8,camel,62,68
9,Case,68,72


## Featurization

For many use cases, the text of tokens is sufficient for further processing; for others, capturing features for each token is desired.

* Character-level features are directly available from the feature matrix.
* Token features can be expressed
    * **_directly_** as the sum of all character features for the token
        * These amount to "characteristic" vectorization of tokens, where vectors for all tokens having the same combination of character features are equivalent.
    * **_abstractly_** as the labeled combination of multiple character features
        * **NOTE:** These are typically labels given to the offset combinations used to split text into tokens in the first place, but can include the use of other rules, like regular expression (in)validation, over the token data.

### Direct token featurization

Show the direct characteristic vector for the featurized tokens.

In [6]:
print(f'Featurized tokenization of text:\n\n{text}\n')

data2 = [(token.text,
          token.start_idx, token.end_idx,
          ' '.join(str(f) for f in token.features))
         for token in la_tokens]
df = pd.DataFrame(data2, columns=['token', 'start_idx', 'end_idx', 'characteristic_vector'])
df

Featurized tokenization of text:

This is a1 test don't http://foo.com?bar=123 @user abc@xyz.com camelCaseOne, CamelCaseTwo, camelCase1, CamelCase2, 123 $123,456.78



,token,start_idx,end_idx,characteristic_vector
0,This,0,4,4 4 0 3 1 0 0 0 0 0 0 0 3 3 3 3 2 3 1 1 0 0 0 ...
1,is,4,7,2 2 0 2 0 1 0 0 0 0 0 0 2 2 2 2 2 2 1 1 0 0 0 ...
2,a1,7,10,1 2 1 1 0 1 0 0 0 0 0 0 2 1 2 2 2 1 1 1 0 0 0 ...
3,test,10,15,4 4 0 4 0 1 0 0 0 0 0 0 3 4 4 4 3 4 1 1 0 0 0 ...
4,don't,15,21,4 4 0 4 0 1 1 0 0 0 0 0 4 4 4 4 4 4 1 1 1 0 0 ...
5,http://foo.com?bar=123,21,44,13 16 3 13 0 1 6 0 0 1 2 1 14 13 16 16 14 13 1...
6,@user,44,50,4 4 0 4 0 1 1 1 1 0 0 0 3 4 4 4 3 4 1 1 1 1 0 ...
7,abc@xyz.com,50,62,9 9 0 9 0 1 2 1 1 0 0 1 9 9 9 9 9 9 1 1 2 1 0 ...
8,camel,62,68,5 5 0 5 0 1 0 0 0 0 0 0 5 6 5 6 5 5 1 0 0 0 0 ...
9,Case,68,72,4 4 0 3 1 0 0 0 0 0 0 0 4 4 4 4 3 3 0 0 0 0 0 ...


### Abstract token featurization

Show the abstract feature specs to the token objects.

In [7]:
print(f'Abstract featurization of text:\n\n{text}\n')

data3 = [(token.text,
          token.start_idx, token.end_idx,
          token.abstract_features)
         for token in la_tokens]
df = pd.DataFrame(data3, columns=['token', 'start_idx', 'end_idx', 'abstract_features'])
df

Abstract featurization of text:

This is a1 test don't http://foo.com?bar=123 @user abc@xyz.com camelCaseOne, CamelCaseTwo, camelCase1, CamelCase2, 123 $123,456.78



,token,start_idx,end_idx,abstract_features
0,This,0,4,None
1,is,4,7,None
2,a1,7,10,[numeric]
3,test,10,15,None
4,don't,15,21,[apos]
5,http://foo.com?bar=123,21,44,"[url, numeric]"
6,@user,44,50,[twitter]
7,abc@xyz.com,50,62,[email]
8,camel,62,68,None
9,Case,68,72,[camelcase]
